In [ ]:
##THIS SCRIPT CONVERTS DAILY HEATFLUX TO DAILY SEA ICE PRODUCTION
#USES SIC IN CALC, AND TO MASK OUT NON POLYNYA AREAS

#SET:
# - path to folder for output tifs
# - path to SIC files
# - path to HeatFlux data

import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import gdal

#outputfolder =''
path = '//' #Path to masked SIC tifs
noOfImages =  #set

dataset= gdal.Open('.tif') #Path to calculated heatflux tif
HeatFluxAll = dataset.ReadAsArray()


file_list2 = [x for x in sorted([x for x in os.listdir(path)])]
thing = '.DS_Store' #Hidden file that occurs. 
while thing in file_list2: file_list2.remove(thing)

image_frames = []
days = np.arange(0,) #fix - use noOfImages?

for k in days:

    image = Image.open(path + file_list2[(days[k])]) #Cycles through file list. 
    Y = np.array(image)
    Y = Y.astype("float") #Necessary to convert to Nan
    #Y[Y == 255] = np.nan #Convert nodata to NaN
    ##Get Heatflux
    Heatflux = HeatFluxAll[k,:,:] #assuming start from first band. k-1 cos first band indexed as 0, k starts at 1
    Iceproduction = (3125*3125)*(1-(Y/2/100))*Heatflux/(920*(2.79*10**5)) #Removed 84600* (seconds in day)
    # Try an example pixel of HeatFluxValue 25135448
    
    ##Make mask from SIC #Conveluted way of making values that are 0 to 0.7 = to 1, and all values >0.7 incl nodata = 0
    SICO_mask = Y/2/100
    SICO_mask[SICO_mask == 0] = 0.1 
    SICO_mask[SICO_mask > 0.7] = 0 
    SICO_mask[(SICO_mask <= 0.7)&(SICO_mask>0)] = 1  
    
    ##Apply mask to iceproduction
    Iceproduction = SICO_mask*Iceproduction
    
    #no data value
    Iceproduction[Iceproduction == -4.83591813831807e+35] = 0 #-4.83591813831807e+35 is no data value

    x = np.sum(Iceproduction)
    #print(k)
    
    #print(x*1e-9) #print in km3 -- CHOOSE WHICH
    #print(SICO_mask.sum()*(3.125**2)) #print area in km2 -- CHOOSE WHICH
    #print(x)

    
    